In [1]:
import sys 
sys.path.append("..") 
import pickle

import pandas as pd
import matplotlib.pyplot as plt

from utils import data_helper,result_helper
from models import lstm_model

In [2]:
train_ds2='../data/train/unlabeled_data_labeled.csv'
train_df2=pd.read_csv(train_ds2)
train_df2

,content,class_label,by_word
0,忍者学园中玩家们可以进行PVE模式的刷图，而在副本中也是有着很多的BOSS在等着玩家们去攻略...,游戏,玩家
1,多多自走棋最近也是上新了新的棋子，而本次的这个棋子是灵族战士的岩之灵，这是一个定位为肉盾和控...,游戏,玩家
2,在不休的乌拉拉中有不少小伙伴是很喜欢斗士这个职业的，有爆发有回血，而且操作空间很大，而在玩斗...,游戏,玩家
3,明日方舟最近也是开启了最新的活动，这次的活动叫做是-午间逸话，不过很多玩家不知道这个最后一关...,游戏,玩家
4,侍魂胧月传说在今天12月4日迎来了一波维护更新，值得一提的是在本次更新以后就将会迎来周年庆哦...,游戏,玩家
...,...,...,...
13171,近日有关于科普下踢足球技巧足球比赛怎么做好战术配合的问题受到了很多网友们的关注，大多数网友都...,体育,队员
13172,2020年高考今日开考，湖北近40万考生奔赴考场(2020湖北高考语文作文题：齐桓公管仲鲍叔...,体育,队员
13173,棒球可能对很多中国人来说都还有点陌生，其实棒球也是一项很好的休闲运动，也受到了很多人的喜欢，...,体育,队员
13174,­ 儿子与母亲做生意有矛盾，居然欺骗母亲，把母亲卡上的107万给转走了！逼使母亲拿起法律的...,体育,队员


In [3]:
train_ds1='../data/train/labeled_data_processed.csv'
train_df1=pd.read_csv(train_ds1)
train_df1

,id,class_label,content,text_len,label,cleaned_content,cutted_content
0,0,教育,人民网北京8月31日电（孙竞）日前，为进一步规范高等学校命名工作，依据相关法律法规规定，结合...,692,2,人民网北京8月31日电孙竞日前为进一步规范高等学校命名工作依据相关法律法规规定结合高校设置工...,人民网 北京 8 月 31 日电 孙竞 日前 进一步 规范 高等学校 命名 工作 依据 相关...
1,1,家居,又是一年落叶黄，一阵秋雨一阵凉；整日奔波工作忙，出门别忘添衣裳。金秋时节，正值装修旺季，集美...,816,0,又是一年落叶黄一阵秋雨一阵凉整日奔波工作忙出门别忘添衣裳金秋时节正值装修旺季集美家居继续带消...,一年 落叶 黄 一阵 秋雨 一阵 凉 整日 奔波 工作 忙 出门 别忘添 衣裳 金秋 时节 ...
2,2,教育,作者：祝安顺（深圳大学饶宗颐文化研究院编审、副教授） 越来越多的名校毕业生入职教师行业，吸...,1379,2,作者祝安顺深圳大学饶宗颐文化研究院编审副教授越来越多的名校毕业生入职教师行业吸引他们的是什么...,作者 祝 安顺 深圳大学 饶宗 颐 文化 研究院 编审 副教授 越来越 名校 毕业生 入职 ...
3,3,时尚,虽然此前其有过与品牌如Madawell的合作系列，但这回她确认将推出自己名下的服装系列。在最...,232,3,虽然此前其有过与品牌如Madawell的合作系列但这回她确认将推出自己名下的服装系列在最近的...,此前 其有 品牌 Madawell 合作 系列 这回 确认 推出 名下 服装 系列 采访 中...
4,4,教育,眼下，虽然疫情依然严峻，但我国大中小学都已陆续开学，疫情防控进入常态化，学生校园生活回归正常...,1740,2,眼下虽然疫情依然严峻但我国大中小学都已陆续开学疫情防控进入常态化学生校园生活回归正常尽管国内...,眼下 疫情 依然 严峻 我国 大中小学 都 已 陆续 开学 疫情 防控 常态 化 学生 校园...
...,...,...,...,...,...,...,...
6995,6995,教育,近期，国家卫健委发布《学校传染病症状监测预警技术指南》，其中规定，学校传染病达预警指标后，应...,1354,2,近期国家卫健委发布学校传染病症状监测预警技术指南其中规定学校传染病达预警指标后应立即对出现症...,近期 国家 卫健委 发布 学校 传染病 症状 监测 预警 技术 指南 学校 传染病 达 预警...
6996,6996,房产,新冠肺炎疫情影响之下，房企纷纷开设“线上售楼处”。据克而瑞监测，目前超过150家房企在做线上...,2819,1,新冠肺炎疫情影响之下房企纷纷开设线上售楼处据克而瑞监测目前超过150家房企在做线上营销其中超...,新冠 肺炎 疫情 影响 之下 房企 纷纷 开设 线上 售楼处 据克而瑞 监测 超过 150 ...
6997,6997,教育,学生犯错后，教师能不能惩戒？如何惩戒？惩戒的边界在哪里？9月起，《广东省学校安全条例》（以下...,1551,2,学生犯错后教师能不能惩戒如何惩戒惩戒的边界在哪里9月起广东省学校安全条例以下简称条例施行其中...,学生 犯错 后 教师 惩戒 惩戒 惩戒 边界 9 月 广东省 学校 条例 简称 条例 施行 ...
6998,6998,科技,6月3日下午，荣耀为我们带来了全新的荣耀Play4系列新品，包括了Play4和Play4Pr...,520,5,6月3日下午荣耀为我们带来了全新的荣耀Play4系列新品包括了Play4和Play4Pro两...,6 月 3 日 下午 荣耀 带来 全新 荣耀 Play4 系列 新品 包括 Play4 Pl...


In [4]:
train_df2['cutted_content']=data_helper.preprocess_text_series(train_df2['content'],'../data/baidu_stopwords.txt')
train_df2

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\HAGGI-~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.742 seconds.
Prefix dict has been built successfully.


,content,class_label,by_word,cutted_content
0,忍者学园中玩家们可以进行PVE模式的刷图，而在副本中也是有着很多的BOSS在等着玩家们去攻略...,游戏,玩家,忍者 学园 中 玩家 PVE 模式 刷 图 副本 中 很多 BOSS 玩家 去 攻略 女帝 ...
1,多多自走棋最近也是上新了新的棋子，而本次的这个棋子是灵族战士的岩之灵，这是一个定位为肉盾和控...,游戏,玩家,多多 走棋 上 新 新 棋子 本次 棋子 灵族 战士 岩之灵 一个 定位 肉盾 控制 类型 ...
2,在不休的乌拉拉中有不少小伙伴是很喜欢斗士这个职业的，有爆发有回血，而且操作空间很大，而在玩斗...,游戏,玩家,不休 乌拉 拉中 不少 小伙伴 很 喜欢 斗士 职业 爆发 回血 操作 空间 很大 玩 斗士...
3,明日方舟最近也是开启了最新的活动，这次的活动叫做是-午间逸话，不过很多玩家不知道这个最后一关...,游戏,玩家,明日 方舟 开启 最新 活动 这次 活动 午间 逸 话 很多 玩家 不 一关 打小编 爱 带...
4,侍魂胧月传说在今天12月4日迎来了一波维护更新，值得一提的是在本次更新以后就将会迎来周年庆哦...,游戏,玩家,侍魂 胧 月 传说 12 月 4 日 迎来 一波 维护 更新 值得一提的是 本次 更新 会 ...
...,...,...,...,...
13171,近日有关于科普下踢足球技巧足球比赛怎么做好战术配合的问题受到了很多网友们的关注，大多数网友都...,体育,队员,近日 科普 下 踢足球 技巧 足球比赛 做好 战术 配合 很多 网友 关注 网友 都 想要 ...
13172,2020年高考今日开考，湖北近40万考生奔赴考场(2020湖北高考语文作文题：齐桓公管仲鲍叔...,体育,队员,2020 年 高考 今日 开考 湖北 近 40 万 考生 奔赴 考场 2020 湖北 高考 ...
13173,棒球可能对很多中国人来说都还有点陌生，其实棒球也是一项很好的休闲运动，也受到了很多人的喜欢，...,体育,队员,棒球 很多 中国 人 来说 都 还 陌生 棒球 一项 很 好 休闲 运动 很多 人 喜欢 棒...
13174,­ 儿子与母亲做生意有矛盾，居然欺骗母亲，把母亲卡上的107万给转走了！逼使母亲拿起法律的...,体育,队员,儿子 母亲 做生意 矛盾 居然 欺骗 母亲 母亲 卡上 107 万给 转 走 逼使 母亲 法...


In [5]:
class_label_to_class_dict = {
            0: '家居', 1: '房产', 2: '教育', 3: '时尚', 4: '时政', 5: '科技', 6: '财经', 7: '游戏', 8: '娱乐', 9: '体育'}

class_to_class_label_dict={val:key for key,val in class_label_to_class_dict.items()}
class_to_class_label_dict

{'家居': 0,
 '房产': 1,
 '教育': 2,
 '时尚': 3,
 '时政': 4,
 '科技': 5,
 '财经': 6,
 '游戏': 7,
 '娱乐': 8,
 '体育': 9}

In [6]:
train_df2['label']=train_df2['class_label'].apply(lambda x:class_to_class_label_dict[x])
train_df2

,content,class_label,by_word,cutted_content,label
0,忍者学园中玩家们可以进行PVE模式的刷图，而在副本中也是有着很多的BOSS在等着玩家们去攻略...,游戏,玩家,忍者 学园 中 玩家 PVE 模式 刷 图 副本 中 很多 BOSS 玩家 去 攻略 女帝 ...,7
1,多多自走棋最近也是上新了新的棋子，而本次的这个棋子是灵族战士的岩之灵，这是一个定位为肉盾和控...,游戏,玩家,多多 走棋 上 新 新 棋子 本次 棋子 灵族 战士 岩之灵 一个 定位 肉盾 控制 类型 ...,7
2,在不休的乌拉拉中有不少小伙伴是很喜欢斗士这个职业的，有爆发有回血，而且操作空间很大，而在玩斗...,游戏,玩家,不休 乌拉 拉中 不少 小伙伴 很 喜欢 斗士 职业 爆发 回血 操作 空间 很大 玩 斗士...,7
3,明日方舟最近也是开启了最新的活动，这次的活动叫做是-午间逸话，不过很多玩家不知道这个最后一关...,游戏,玩家,明日 方舟 开启 最新 活动 这次 活动 午间 逸 话 很多 玩家 不 一关 打小编 爱 带...,7
4,侍魂胧月传说在今天12月4日迎来了一波维护更新，值得一提的是在本次更新以后就将会迎来周年庆哦...,游戏,玩家,侍魂 胧 月 传说 12 月 4 日 迎来 一波 维护 更新 值得一提的是 本次 更新 会 ...,7
...,...,...,...,...,...
13171,近日有关于科普下踢足球技巧足球比赛怎么做好战术配合的问题受到了很多网友们的关注，大多数网友都...,体育,队员,近日 科普 下 踢足球 技巧 足球比赛 做好 战术 配合 很多 网友 关注 网友 都 想要 ...,9
13172,2020年高考今日开考，湖北近40万考生奔赴考场(2020湖北高考语文作文题：齐桓公管仲鲍叔...,体育,队员,2020 年 高考 今日 开考 湖北 近 40 万 考生 奔赴 考场 2020 湖北 高考 ...,9
13173,棒球可能对很多中国人来说都还有点陌生，其实棒球也是一项很好的休闲运动，也受到了很多人的喜欢，...,体育,队员,棒球 很多 中国 人 来说 都 还 陌生 棒球 一项 很 好 休闲 运动 很多 人 喜欢 棒...,9
13174,­ 儿子与母亲做生意有矛盾，居然欺骗母亲，把母亲卡上的107万给转走了！逼使母亲拿起法律的...,体育,队员,儿子 母亲 做生意 矛盾 居然 欺骗 母亲 母亲 卡上 107 万给 转 走 逼使 母亲 法...,9


In [7]:
# 打乱数据
train_df2=train_df2.sample(frac=1).reset_index(drop=True)
train_df2

,content,class_label,by_word,cutted_content,label
0,在雪鹰领主中，法宝是非常重要的攻略，而每个法宝都会有着洗练的六种属性，而洗练法宝也是提升玩家...,游戏,玩家,雪鹰 领主 中 法宝 攻略 法宝 都 会 洗练 六种 属性 洗练 法宝 提升 玩家 战斗力 ...,7
1,受新冠肺炎疫情影响推迟了大半个赛季的世界田联钻石联赛终于迎来了回归，在14日进行的揭幕战摩纳...,体育,运动员,受新冠 肺炎 疫情 影响 推迟 大半个 赛季 世界 田联 钻石 联赛 终于 迎来 回归 14...,9
2,明星闪耀Valentino虚拟博物馆作者：韩梅时间：2011年12月21日内容来源：VOGU...,娱乐,明星,明星 闪耀 Valentino 虚拟 博物馆 作者 韩梅 时间 2011 年 12 月 21...,8
3,圣诞节前夕，西方主流媒体频频发表批评中国政府“压制宗教自由”的文章。有的西方网站贴出中国“内...,娱乐,媒体,圣诞节 前夕 西方 主流 媒体 频频 发表 批评 中国政府 压制 宗教自由 文章 西方 网站...,8
4,2021年环法自行车赛将比原计划提前一周开始，为了避免与东京奥运会开幕时间重叠。据国际自行车...,体育,奥运会,2021 年 环法 自行车赛 比原 计划 提前 一周 东京 奥运会 开幕 时间重叠 国际 自...,9
...,...,...,...,...,...
13171,7月5日，古装魔幻电影《封魔传》在横店正式开机，导演苗述携陈键锋、于函冰、朱茗義、张鑫娜等主...,娱乐,导演,7 月 5 日 古装 魔幻 电影 封魔传 横店 正式 开机 导演 苗述 携 陈键锋 函冰 朱...,8
13172,很多玩影之刃3的萌新玩家总觉得自己提升的太慢了，但是自己又不是氪金大佬就很烦恼，其实不用氪金...,游戏,玩家,很多 玩影 之刃 3 萌新 玩家 总 提升 太慢 氪 金 大佬 很 烦恼 不用 氪 金 快速...,7
13173,山海镜花游戏中上线了一段时间了，自从开服以来，游戏中的玩家也一直不断的增加中，所以新手玩家们...,游戏,玩家,山海 镜花 游戏 中 上线 一段时间 开服 游戏 中 玩家 增加 中 新手 玩家 都 问 山...,7
13174,近日，外媒曝光的艾梅伯·希尔德和约翰尼德普关于家暴的对话录音造成了轰动，在录音里，艾梅伯承认...,游戏,游戏,近日 外媒 曝光 艾梅伯 希尔德 约翰尼 德普 家暴 对话 录音 轰动 录音 里 艾梅伯 承...,7


In [9]:
# 后三类每个保留1000条数据
train_df2_sub=pd.DataFrame()
for label in train_df2['label'].unique():
    train_df2_sub=train_df2_sub.append(train_df2[train_df2['label']==label].iloc[:1000])
train_df2_sub

,content,class_label,by_word,cutted_content,label
0,在雪鹰领主中，法宝是非常重要的攻略，而每个法宝都会有着洗练的六种属性，而洗练法宝也是提升玩家...,游戏,玩家,雪鹰 领主 中 法宝 攻略 法宝 都 会 洗练 六种 属性 洗练 法宝 提升 玩家 战斗力 ...,7
6,7月17日上午，@拯救者电竞手机正式宣布，拯救者电竞手机Pro将是全球首款采用中置弹出式的手...,游戏,玩法,7 月 17 日 上午 拯救 电 竞 手机 正式 拯救 电 竞 手机 Pro 全球 首款 采...,7
13,战斗天赋解析系统中有很多的角色，这些角色的属性都是非常给力的，这里给大家带来其中的这个可可具...,游戏,玩法,天赋 解析 系统 中有 很多 角色 角色 属性 都 给力 带来 可可 介绍 想 可可 玩法 ...,7
19,今天下午三点，腾讯黑鲨游戏手机3S将正式和大家见面，可以说是游戏党们又多了一个可以选择的上分...,游戏,玩家,今天下午 三点 腾讯 黑 鲨 游戏 手机 3S 正式 见面 说 游戏 党们 一个 选择 上分...,7
20,灌篮高手中有很多经典的原著球员，而每个球员都是有着独特还原的技能，而且还都是大招的强力技能，...,游戏,游戏,灌篮高手 中有 很多 经典 原著 球员 球员 都 独特 还原 技能 还 都 大招 强力 技能...,7
...,...,...,...,...,...
1913,"2020中国(顺德)家电博览会已于8月3在广东潭洲国际会展中心隆重开幕。本届展会为期三天,期...",娱乐,媒体,2020 中国 顺德 家电 博览会 已于 8 月 3 广东 潭洲 国际 会展中心 隆重开幕 ...,8
1915,11月24日，北京四环外，油门轰鸣，机车集结，黑衣骑士们共赴一场盛宴。即将于11月30日上映...,娱乐,粉丝,11 月 24 日 北京 四环 外 油门 轰鸣 机车 集结 黑衣 骑士 共赴 一场 盛宴 即...,8
1917,近日，由美国哥伦比亚影片公司出品的超级英雄巨制《蜘蛛侠：英雄远征》在京举办了盛况空前的新闻发...,娱乐,粉丝,近日 美国 哥伦比亚 影片 公司出品 超级 英雄 巨制 蜘蛛侠 英雄 远征 京 举办 盛况空...,8
1919,由王小帅执导，王景春、咏梅领衔主演，齐溪、王源、杜江等主演的电影《地久天长》即将于3月22日...,娱乐,粉丝,王小帅 执导 王景春 咏梅 领衔主演 齐溪 王源 杜江 主演 电影 地久天长 即将 3 月 ...,8


In [13]:
train_df=train_df1[['cutted_content','label']].append(train_df2_sub[['cutted_content','label']])
trian_df

,cutted_content,label
0,人民网 北京 8 月 31 日电 孙竞 日前 进一步 规范 高等学校 命名 工作 依据 相关...,2
1,一年 落叶 黄 一阵 秋雨 一阵 凉 整日 奔波 工作 忙 出门 别忘添 衣裳 金秋 时节 ...,0
2,作者 祝 安顺 深圳大学 饶宗 颐 文化 研究院 编审 副教授 越来越 名校 毕业生 入职 ...,2
3,此前 其有 品牌 Madawell 合作 系列 这回 确认 推出 名下 服装 系列 采访 中...,3
4,眼下 疫情 依然 严峻 我国 大中小学 都 已 陆续 开学 疫情 防控 常态 化 学生 校园...,2
...,...,...
1913,2020 中国 顺德 家电 博览会 已于 8 月 3 广东 潭洲 国际 会展中心 隆重开幕 ...,8
1915,11 月 24 日 北京 四环 外 油门 轰鸣 机车 集结 黑衣 骑士 共赴 一场 盛宴 即...,8
1917,近日 美国 哥伦比亚 影片 公司出品 超级 英雄 巨制 蜘蛛侠 英雄 远征 京 举办 盛况空...,8
1919,王小帅 执导 王景春 咏梅 领衔主演 齐溪 王源 杜江 主演 电影 地久天长 即将 3 月 ...,8


In [15]:
train_df=train_df.sample(frac=1).reset_index(drop=True)
train_df['label'].value_counts()

7    1000
6    1000
5    1000
4    1000
3    1000
2    1000
9    1000
1    1000
8    1000
0    1000
Name: label, dtype: int64

In [16]:
train_df.to_csv('../data/train/train_processed.csv',index=False)